Axioms for context <a class="ProveItLink" href="_context_.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
# the context is in the current directory:
context = proveit.Context('.') # adds context root to sys.path if necessary

In [ ]:
%begin axioms

In [ ]:
%end axioms